<a href="https://colab.research.google.com/github/yazoo1220/AskTwitterAlgo/blob/main/Langchain_SQL_chain_from_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q pandas sqlalchemy langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/LLM/SQL/shokuhin_kcal.csv')

In [ ]:
df

,食品名,kcal/100 g
0,アマランサス 玄穀,358
1,あわ 精白粒,367
2,あわ あわもち,214
3,えんばく オートミール,380
4,おおむぎ 七分つき押麦,341
...,...,...
2186,ハンバーグ 冷凍,223
2187,ピラフ 冷凍,161
2188,ミートボール 冷凍,244
2189,メンチカツ フライ用 冷凍,196


In [ ]:
from sqlalchemy import create_engine

# Create SQLite engine
engine = create_engine('sqlite:///shokuhin.db')

# Store DataFrame to sqlite
df.to_sql('shokuhin', engine)

2191

In [ ]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR_API_KEY'

In [ ]:
db = SQLDatabase.from_uri("sqlite:///shokuhin.db")
llm = OpenAI(temperature=0, verbose=True)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True) #, return_direct=True)

'\n豚肉: 190g\nにんじん: 80g\n玉ねぎ: 50g\nじゃがいも: 80g\nカレールー: 115g\n水: 830ml\nサラダ油: 適量'

In [ ]:
recipe='''
米
180g
玉ねぎ
1/4個
薄切りハーフベーコン
60g
ミックスベジタブル (冷凍)
20g
(A)ケチャップ
大さじ2
(A)中濃ソース
大さじ1
(A)オリーブオイル
大さじ1
卵
有塩バター
10g
(B)卵
3個
(B)生クリーム
30ml
(B)塩
少々
ケチャップ
大さじ1
パセリ (生)
適量
'''
query = llm('単位を「本」「個」などからgに変換してください。ごはんは「米」など日本食品標準成分表にあわせた表記にしてください。適量、などのようなあいまいなものは無視してください。' + recipe)
print(query)
db_chain.run(query)


米 180g
玉ねぎ 1/4個
薄切りハーフベーコン 60g
ミックスベジタブル (冷凍) 20g
(A)ケチャップ 大さじ2
(A)中濃ソース 大さじ1
(A)オリーブオイル 大さじ1
卵 3個
有塩バター 10g
(B)生クリーム 30ml
(B)塩 少々
ケチャップ 大さじ1
パセリ (生) 適量


> Entering new SQLDatabaseChain chain...

米 180g
玉ねぎ 1/4個
薄切りハーフベーコン 60g
ミックスベジタブル (冷凍) 20g
(A)ケチャップ 大さじ2
(A)中濃ソース 大さじ1
(A)オリーブオイル 大さじ1
卵 3個
有塩バター 10g
(B)生クリーム 30ml
(B)塩 少々
ケチャップ 大さじ1
パセリ (生) 適量
SQLQuery:SELECT SUM("kcal/100 g") 
FROM shokuhin 
WHERE "食品名" IN ('米', '玉ねぎ', 'ハーフベーコン', 'ミックスベジタブル', '(A)ケチャップ', '(A)中濃ソース', '(A)オリーブオイル', '卵', '有塩バター', '(B)生クリーム', '(B)塩', 'ケチャップ', 'パセリ')
SQLResult: [(None,)]
Answer:The total kcal/100 g of the ingredients is not available in the database.
> Finished chain.


'The total kcal/100 g of the ingredients is not available in the database.'